In [16]:
# 필요 패키지 설치
# !pip install requests geopy pyproj dask folium contextily aiohttp tqdm scikit-learn sqlalchemy
# !pip install polars

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

import requests
import time

# 선택
import geopandas as gpd
from shapely.geometry import Point

In [30]:
apt2013_df = pd.read_csv('Data_Preprocessing/add_2013.csv', encoding="utf-8")
apt2014_df = pd.read_csv('Data_Preprocessing/add_2014.csv', encoding="utf-8")
apt2015_df = pd.read_csv('Data_Preprocessing/add_2015.csv', encoding="utf-8")
apt2016_df = pd.read_csv('Data_Preprocessing/add_2016.csv', encoding="utf-8")
apt2017_df = pd.read_csv('Data_Preprocessing/add_2017.csv', encoding="utf-8")
apt2018_df = pd.read_csv('Data_Preprocessing/add_2018.csv', encoding="utf-8")
apt2019_df = pd.read_csv('Data_Preprocessing/add_2019.csv', encoding="utf-8")
apt2020_df = pd.read_csv('Data_Preprocessing/add_2020.csv', encoding="utf-8")
apt2021_df = pd.read_csv('Data_Preprocessing/add_2021.csv', encoding="utf-8")
apt2022_df = pd.read_csv('Data_Preprocessing/add_2022_원본.csv', encoding="utf-8")
apt2023_df = pd.read_csv('Data_Preprocessing/add_2023.csv', encoding="utf-8")
apt2024_df = pd.read_csv('Data_Preprocessing/add_2024.csv', encoding="utf-8")

In [31]:
print(apt2013_df.info())
print(apt2014_df.info())
print(apt2015_df.info())
print(apt2016_df.info())
print(apt2017_df.info())
print(apt2018_df.info())
print(apt2019_df.info())
print(apt2020_df.info())
print(apt2021_df.info())
print(apt2022_df.info())
print(apt2023_df.info())
print(apt2024_df.info())
print(apt2013_df.head(2))
print(apt2014_df.head(2))
print(apt2015_df.head(2))
print(apt2016_df.head(2))
print(apt2017_df.head(2))
print(apt2018_df.head(2))
print(apt2019_df.head(2))
print(apt2020_df.head(2))
print(apt2021_df.head(2))
print(apt2022_df.head(2))
print(apt2023_df.head(2))
print(apt2024_df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68207 entries, 0 to 68206
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NO      68207 non-null  int64 
 1   기존주소    68207 non-null  object
 2   도로명주소   68207 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85644 entries, 0 to 85643
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NO      85644 non-null  int64 
 1   기존주소    85644 non-null  object
 2   도로명주소   85644 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120162 entries, 0 to 120161
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   NO      120162 non-null  int64 
 1   기존주소    120162 non-null  object
 2   도로명주소   120162 non-null  object
dtypes: int64(1), o

In [32]:
# # Kakao API Geocoding 함수
# def geocode_kakao(address, api_key):
#     """
#     Kakao Local API를 사용해 주소를 위도, 경도로 변환하는 함수
#     :param address: 도로명 주소
#     :param api_key: Kakao REST API Key
#     :return: (latitude, longitude) 좌표 튜플
#     """
#     url = "https://dapi.kakao.com/v2/local/search/address.json"
#     headers = {"Authorization": f"KakaoAK {api_key}"}
#     params = {"query": address}
    
#     response = requests.get(url, headers=headers, params=params)
    
#     if response.status_code == 200:
#         result = response.json()
#         if result['documents']:
#             x = float(result['documents'][0]['x'])  # 경도 (longitude)
#             y = float(result['documents'][0]['y'])  # 위도 (latitude)
#             return y, x
#         else:
#             return None, None  # 결과 없음
#     else:
#         print(f"Error {response.status_code}: {response.text}")
#         return None, None

# # 주소 리스트가 담긴 데이터프레임 예시
# # apt2013_df ~ apt2024_df를 하나로 합치거나, 필요한 데이터프레임에서 '도로명주소'를 가져옵니다.
# # 모든 데이터프레임을 합친 예시:
# all_addresses = pd.concat([apt2013_df, apt2014_df, apt2015_df, apt2016_df, apt2017_df, 
#                            apt2018_df, apt2019_df, apt2020_df, apt2021_df, apt2022_df, 
#                            apt2023_df, apt2024_df])['도로명주소'].drop_duplicates()

# # 결과 저장할 리스트
# coordinates = []

# # API Key 설정
# kakao_api_key = "81f44d821f5434c7657e027aae509fbb"  # Kakao REST API 키를 입력하세요

# # 지오코딩 실행
# for idx, address in enumerate(all_addresses):
#     lat, lng = geocode_kakao(address, kakao_api_key)
#     coordinates.append({'address': address, 'latitude': lat, 'longitude': lng})
    
#     # 진행 상황 출력
#     if idx % 100 == 0:
#         print(f"{idx}/{len(all_addresses)} addresses processed.")
    
#     # API 호출 제한을 피하기 위해 딜레이 추가 (필요시)
#     time.sleep(0.2)  # 0.2초 대기 (초당 5건 요청)

# # 결과를 데이터프레임으로 변환
# coordinates_df = pd.DataFrame(coordinates)

# # 결과 저장
# coordinates_df.to_csv("geocoded_addresses.csv", index=False)


In [33]:
apt2013_df
apt2014_df
apt2015_df
apt2016_df
apt2017_df
apt2018_df
apt2019_df
apt2020_df
apt2021_df
apt2022_df
apt2023_df
apt2024_df

,NO,기존주소,도로명주소
0,1,서울특별시 광진구 자양동 695,서울특별시 광진구 구의강변로 11
1,2,서울특별시 금천구 독산동 1147,서울특별시 금천구 벚꽃로 40
2,3,서울특별시 동작구 대방동 502,서울특별시 동작구 등용로 127
3,4,서울특별시 중랑구 중화동 450,서울특별시 중랑구 동일로 752
4,5,서울특별시 마포구 창전동 443,서울특별시 마포구 토정로 167
...,...,...,...
50564,50565,서울특별시 도봉구 쌍문동 731,서울특별시 도봉구 노해로41가길 16
50565,50566,서울특별시 송파구 가락동 53,서울특별시 송파구 송이로21길 23
50566,50567,서울특별시 송파구 가락동 165,서울특별시 송파구 동남로 225
50567,50568,서울특별시 강동구 상일동 519,서울특별시 강동구 고덕로 360


In [34]:
time.localtime().tm_year, time.localtime().tm_mon, time.localtime().tm_mday,  

(2024, 11, 26)

In [35]:
eval(f'apt{2013}_df')

,NO,기존주소,도로명주소
0,1,서울특별시 동대문구 전농동 687,서울특별시 동대문구 서울시립대로 75
1,2,서울특별시 용산구 보광동 448,서울특별시 용산구 보광로 14
2,3,서울특별시 용산구 서빙고동 241-21,서울특별시 용산구 이촌로 347
3,4,서울특별시 성동구 송정동 90,서울특별시 성동구 동일로 263
4,5,서울특별시 성북구 정릉동 1018,서울특별시 성북구 서경로2길 48
...,...,...,...
68202,68203,서울특별시 구로구 구로동 1280,서울특별시 구로구 디지털로31길 120
68203,68204,서울특별시 중구 신당동 851,서울특별시 중구 퇴계로90길 74
68204,68205,서울특별시 동대문구 장안동 329-3,서울특별시 동대문구 장안벚꽃로 167
68205,68206,서울특별시 은평구 응암동 176,서울특별시 은평구 백련산로 146


In [36]:
print('13:', len(apt2013_df) )
print('14:', len(apt2014_df) )
print('15:', len(apt2015_df) )
print('16:', len(apt2016_df) )
print('17:', len(apt2017_df) ) # 뒷자리
print('17:', len(apt2018_df) ) # 뒷자리
print('19:', len(apt2019_df) ) # 내옆
print('20:', len(apt2020_df) ) # 원빈씨
print('21:', len(apt2021_df) ) # 선생님
print('22:', len(apt2022_df) ) # 수연씨
print('23:', len(apt2023_df) ) # 선생님

len(apt2019_df) + len(apt2020_df) +len(apt2021_df) + len(apt2022_df)+len(apt2023_df)
# len(apt2024_df)
# print('19~23년 총 개수:', )

13: 68207
14: 85644
15: 120162
16: 110246
17: 105077
17: 81637
19: 75097
20: 84135
21: 43423
22: 12799
23: 35642


251096